# Pandas Series: apply(), where() ir palyginimas su numpy.where()

Šiame faile pateikiami `pandas.Series` pavyzdžiai:
- `apply()` metodas
- `where()` metodas
- `where()` "grandinimas" (chaining)
- `numpy.where()` ir `pandas.Series.where()` palyginimas

Pavyzdžiai susieti su duomenų analitikos situacijomis ir pateikiami paprastai.

In [ ]:
import pandas as pd
import numpy as np

## 0. Pavyzdiniai duomenys

Žemiau pateikiami keli `Series`, imituojantys produkto analitikos situacijas:
- pajamos (EUR)
- marža (procentais)
- produktų pavadinimai

In [14]:
product = pd.Series(
    ["Protein Bar", "Sparkling Water", "Coffee Beans", "Granola", "USB Cable", "Notebook"],
    index=["P01", "P02", "P03", "P04", "P05", "P06"]
)

revenue = pd.Series([298.128, 300.9902, 712.9840, 496.0515, 1650.0155, 280.1550], index=product.index)
margin = pd.Series([0.42, 0.28, 0.35, 0.31, 0.18, 0.40], index=product.index)

print("product:\n", product, "\n")
print("revenue:\n", revenue, "\n")
print("margin:\n", margin)

product:
 P01        Protein Bar
P02    Sparkling Water
P03       Coffee Beans
P04            Granola
P05          USB Cable
P06           Notebook
dtype: object 

revenue:
 P01     298.1280
P02     300.9902
P03     712.9840
P04     496.0515
P05    1650.0155
P06     280.1550
dtype: float64 

margin:
 P01    0.42
P02    0.28
P03    0.35
P04    0.31
P05    0.18
P06    0.40
dtype: float64


## 1. `apply()` metodas

`apply()` pritaiko funkciją kiekvienam `Series` elementui.

Dažnos paskirtys:
- paprasta transformacija (pvz., suapvalinimas, formatavimas)
- taisyklė (pvz., klasifikavimas)
- nestandartinis valymas (pvz., teksto transformavimas)

Geroji praktika:
- jei įmanoma, pirmiausia rinktis vektorizuotus metodus (`str`, aritmetika, `where`, `clip`),
  nes jie dažniausiai yra aiškesni ir greitesni už `apply()`.

In [15]:
# Pavyzdys: pajamų suapvalinimas iki 2 skaitmenų po kablelio
rev_rounded = revenue.apply(lambda x: round(x, 2))
print(rev_rounded)

P01     298.13
P02     300.99
P03     712.98
P04     496.05
P05    1650.02
P06     280.15
dtype: float64


In [16]:
# Pavyzdys: pajamų segmentavimas į 3 lygius
def revenue_bucket(x):
    if x < 300:
        return "Low"
    elif x <= 700:
        return "Mid"
    else:
        return "High"

rev_segment = revenue.apply(revenue_bucket)
print(rev_segment)

P01     Low
P02     Mid
P03    High
P04     Mid
P05    High
P06     Low
dtype: object


### Dažna klaida: `apply()` naudojimas ten, kur pakanka paprastos vektorizacijos

Jei reikia paprasto skaičiavimo (pvz., pridėti PVM, taikyti koeficientą), `apply()` nėra reikalingas.

In [17]:
# Nereikalingas apply: tas pats rezultatas gaunamas paprasta aritmetika
vat = 0.21

rev_with_vat_fast = revenue * (1 + vat)  # vektorizuota
rev_with_vat_apply = revenue.apply(lambda x: x * (1 + vat))  # veikia, bet nereikalinga

print("Vektorizuota (pirma reikšmė):", round(rev_with_vat_fast.iloc[0], 2))
print("Su apply (pirma reikšmė):", round(rev_with_vat_apply.iloc[0], 2))

Vektorizuota (pirma reikšmė): 360.73
Su apply (pirma reikšmė): 360.73


## 2. `where()` metodas

`Series.where(cond, other=...)` palieka reikšmes ten, kur sąlyga `True`,
o kitur jas pakeičia į `other` (pagal nutylėjimą į `NaN`).

Tai patogu, kai reikia:
- „užmaskuoti“ nepageidaujamas reikšmes
- išlaikyti `Series` ilgį ir indeksą, bet išvalyti arba pakeisti dalį reikšmių

In [18]:
# Pajamos paliekamos tik ten, kur jos >= 300, kitu atveju reikšmė tampa NaN
rev_ge_300 = revenue.where(revenue >= 300)
print(rev_ge_300)

P01          NaN
P02     300.9902
P03     712.9840
P04     496.0515
P05    1650.0155
P06          NaN
dtype: float64


In [19]:
# Pajamos < 300 pakeičiamos į 0, o kitos paliekamos
rev_floor = revenue.where(revenue >= 300, other=0)
print(rev_floor)

P01       0.0000
P02     300.9902
P03     712.9840
P04     496.0515
P05    1650.0155
P06       0.0000
dtype: float64


### Geroji praktika: aiškiai nuspręsti, ar `NaN` yra tinkamas rezultatas

`where` pagal nutylėjimą grąžina `NaN` ten, kur sąlyga netenkinta.
Tai dažnai yra naudinga, nes `NaN` aiškiai žymi, kad reikšmė atmesta.
Jei verslo logika reikalauja 0 arba kitos reikšmės, naudojamas `other=...`.

## 3. `where()` grandinimas (chaining where)

Kai taisyklių yra daugiau, `where()` galima naudoti kelis kartus.
Tai leidžia nuosekliai taikyti kelias filtravimo arba koregavimo sąlygas.

In [25]:
# 1) Atmetamos per mažos pajamos (paliekamos >= 300)
# 2) Iš likusių paliekamos tik tos, kurių marža >= 0.30
rev_filtered = (
    revenue
    .where((revenue >= 300) & (margin >= 0.30))
    #.where(margin >= 0.30)
)

print(rev_filtered)

P01         NaN
P02         NaN
P03    712.9840
P04    496.0515
P05         NaN
P06         NaN
dtype: float64


In [21]:
# Tas pats, bet vietoje NaN naudojama reikšmė 0
rev_filtered_zero = (
    revenue
    .where(revenue >= 300, other=0)
    .where(margin >= 0.30, other=0)
)

print(rev_filtered_zero)

P01      0.0000
P02      0.0000
P03    712.9840
P04    496.0515
P05      0.0000
P06      0.0000
dtype: float64


### Dažna klaida: taisyklės pritaikomos netinkama tvarka

Grandinant `where`, verta pradėti nuo bendresnės taisyklės (pvz., duomenų kokybės),
o tada taikyti specifiškesnes (pvz., verslo kriterijus).
Taip lengviau interpretuoti, kodėl reikšmės „išnyko“.

## 4. `numpy.where()` ir `pandas.Series.where()` palyginimas

Svarbiausi skirtumai:

1) `np.where(cond, a, b)` sukuria rezultatą pagal sąlygą ir visada reikalauja abiejų reikšmių (a ir b).
2) `Series.where(cond, other=...)` yra pandas metodas, kuris išlaiko `Series` indeksą ir pagal nutylėjimą grąžina `NaN` ten, kur sąlyga netenkinta.

Geroji praktika:
- jei tikslas yra „pakeisti dalį reikšmių“, dažnai patogesnis `Series.where()`
- jei reikia greitai sugeneruoti kategorijas („Pass/Fail“), dažnai patogus `np.where()`

In [29]:
# Kategorija pagal pajamų ribą su numpy.where
rev_label_np = np.where(revenue >= 300, "OK", "Low")
rev_label_np = pd.Series(rev_label_np, index=revenue.index)

print("np.where rezultatas:")
print(rev_label_np)

np.where rezultatas:
P01    Low
P02     OK
P03     OK
P04     OK
P05     OK
P06    Low
dtype: object


In [27]:
# Tas pats logiškai, bet su pandas.where (pirmiausia užpildo 'Low', tada palieka 'OK' kur sąlyga True)
rev_label_pd = pd.Series("Low", index=revenue.index).where(revenue < 300, other="OK")

print("pandas.where rezultatas:")
print(rev_label_pd)

pandas.where rezultatas:
P01    Low
P02     OK
P03     OK
P04     OK
P05     OK
P06    Low
dtype: object


### Praktinis palyginimas: indeksas ir suderinimas

`Series.where` natūraliai veikia su indeksu ir gerai dera su kitais pandas objektais.
`np.where` grąžina NumPy masyvą, todėl dažnai verta rezultatą vėl paversti į `Series` su indeksu.

In [30]:
# np.where grąžina numpy masyvą be indekso
raw = np.where(margin >= 0.30, revenue, np.nan)
print("np.where tipas:", type(raw))

# Konvertavimas į Series, kad indeksas būtų išsaugotas
raw_series = pd.Series(raw, index=revenue.index)
print("\nraw_series:")
print(raw_series)

np.where tipas: <class 'numpy.ndarray'>

raw_series:
P01    298.1280
P02         NaN
P03    712.9840
P04    496.0515
P05         NaN
P06    280.1550
dtype: float64


## 5. Trumpa santrauka

- `apply()` pritaiko funkciją kiekvienam elementui, bet dažnai verta pirmiausia ieškoti vektorizuotų alternatyvų.
- `where()` palieka reikšmes, kurios tenkina sąlygą, o kitas pakeičia į `NaN` arba `other` reikšmę.
- `where()` grandinimas leidžia nuosekliai taikyti kelias taisykles.
- `np.where()` patogus greitam kategorijų arba reikšmių sudarymui, tačiau jis nenaudoja indekso, todėl rezultatą dažnai verta paversti į `Series`.